In [1]:
import numpy as np
import pandas as pd

from pprint import pprint
import re

import warnings 
warnings.filterwarnings('ignore')

# NLTK Stop words
from nltk.corpus import stopwords


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# keyword extraction
import yake
from rake_nltk import Rake
from summa import keywords
from keybert import KeyBERT
kw_model = KeyBERT(model='all-mpnet-base-v2')
import nltk

#zero shot classification
from transformers import pipeline

C:\Users\gtull\anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
#df = pd.read_csv(r'queried_datasets_excerpt_cleaned.csv')
#df = pd.read_csv(r'new_data_v1.csv')
df = pd.read_excel(r'new_data.xlsx')

In [39]:
#importing the Stopwords to use them
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','for'])
text = df.excerpt.unique()

In [40]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(text))

print(data_words[5])

['this', 'resource', 'contains', 'presentations', 'from', 'the', 'center', 'for', 'automotive', 'research', 'car', 'management', 'briefing', 'seminars', 'held', 'august', 'with', 'over', 'attendees', 'from', 'industry', 'government', 'media', 'and', 'academia', 'the', 'event', 'featured', 'outstanding', 'presentations', 'from', 'industry', 'thought', 'leaders', 'as', 'well', 'as', 'various', 'networking', 'and', 'social', 'events', 'using', 'car', 'research', 'as', 'foundation', 'these', 'seminars', 'revolved', 'around', 'global', 'manufacturing', 'strategies', 'lightweighting', 'connected', 'vehicles', 'powertrain', 'developments', 'sales', 'forecasting', 'purchasing', 'policy', 'designing', 'for', 'technology', 'and', 'capital', 'investment']


In [41]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=1) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=1)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_lemmatized = process_words(data_words)  # processed Text Data!

In [42]:
print(trigram_mod[bigram_mod[data_lemmatized[0]]])

['know', 'instructor', 'experiential', 'knowledge', 'discipline', 'specific', 'information', 'instruction', 'success', 'rate', 'course', 'intend', 'convey', 'alternative', 'face', 'face', 'instruction', 'aim', 'increase', 'commitment', 'desire', 'faculty', 'utilize', 'online', 'tool', 'instruction', 'user', 'make', 'assumption', 'age', 'gender', 'learner', 'however', 'educational', 'discipline', 'background', 'acknowledge', 'well', 'assumption', 'faculty', 'knowledge', 'effectively', 'operate', 'smart', 'pace', 'study', 'course', 'utilize', 'self', 'pace', 'focused', 'tool', 'well', 'suit', 'accord', 'discipline', 'learner', 'choose', 'commonly', 'utilize', 'tool', 'technology', 'digital', 'source', 'identify', 'technologys', 'support', 'blend', 'fully', 'online', 'environment', 'learner', 'technology', 'evaluation', 'rubric', 'evaluate', 'technology', 'tool', 'select', 'meet', 'specific', 'learn', 'goal', 'learner', 'develop', 'technology', 'integration', 'plan', 'include', 'area', 'n

In [18]:
print(trigram_mod[bigram_mod[data_lemmatized[1]]])

['module', 'consist', 'lab', 'syllabus', 'design', 'enhance', 'automotive', 'electrical', 'course', 'develop', 'seed', 'funding', 'thecaat', 'overview', 'ofhevs', 'thetechnologie', 'power', 'slide', 'area', 'focus', 'basic_theory', 'operation', 'safety', 'service', 'procedure', 'follow', 'discuss', 'type', 'parallel', 'series', 'andserie', 'parallel', 'safety', 'work', 'hevs', 'isolate', 'system', 'disable', 'high', 'voltage', 'circuit', 'high', 'voltage', 'glove', 'insulate', 'tool', 'test', 'hev', 'component', 'ex', 'insulation', 'test', 'electrical', 'difference', 'similar', 'component', 'standard', 'vehicle', 'hevs', 'voltage', 'starter', 'air', 'conditioning', 'transaxle', 'integration', 'power', 'steering', 'regenerative', 'brake', 'correct', 'service', 'procedure', 'battery', 'jump', 'start', 'andmultimet', 'difference', 'type', 'hybrid', 'system', 'compare', 'prius', 'view', 'presentation', 'technician', 'hevs', 'electrical', 'system', 'difference', 'traditional', 'vehicle', 's

In [43]:
#creating a dictionary
gensim_dictionary = corpora.Dictionary(data_lemmatized)

#building a corpus for the topic model
texts = data_lemmatized
gensim_corpus = [gensim_dictionary.doc2bow(text) for text in texts]

In [38]:
#model_results_det = pd.DataFrame(model_results)
model_results_det = pd.read_excel("lda_tuning_results_with_perplexity_251022.xlsx")
model_results_det

,Validation_Set,Topics,Alpha,Beta,Coherence,perplexity
0,75% Corpus,30,0.01,0.01,-17.422493,-82.357875
1,75% Corpus,30,0.01,0.31,-15.879255,-9.481864
2,75% Corpus,30,0.01,0.61,-15.536451,-8.753307
3,75% Corpus,30,0.01,0.91,-14.479821,-8.628451
4,75% Corpus,30,0.01,symmetric,-16.543141,-34.286829
...,...,...,...,...,...,...
595,100% Corpus,39,asymmetric,0.01,-16.956537,-75.989788
596,100% Corpus,39,asymmetric,0.31,-15.523864,-9.277292
597,100% Corpus,39,asymmetric,0.61,-15.765169,-8.708634
598,100% Corpus,39,asymmetric,0.91,-14.443298,-8.585156


In [39]:
model_res_sorted = model_results_det.sort_values('Coherence')
model_res_sorted

,Validation_Set,Topics,Alpha,Beta,Coherence,perplexity
285,75% Corpus,39,0.91,0.01,-19.130098,-81.125605
495,100% Corpus,36,0.91,0.01,-19.059571,-81.001415
555,100% Corpus,38,0.91,0.01,-19.050103,-81.106827
559,100% Corpus,38,0.91,symmetric,-19.049992,-34.846059
585,100% Corpus,39,0.91,0.01,-19.048832,-81.125605
...,...,...,...,...,...,...
133,75% Corpus,34,0.61,0.91,-11.081769,-9.437882
432,100% Corpus,34,0.61,0.61,-11.055159,-9.540531
522,100% Corpus,37,0.61,0.61,-10.993010,-9.603903
103,75% Corpus,33,0.61,0.91,-10.923176,-9.423452


In [40]:
model_res_sorted_per = model_results_det.sort_values('perplexity')
model_res_sorted_per

,Validation_Set,Topics,Alpha,Beta,Coherence,perplexity
240,75% Corpus,38,0.01,0.01,-16.707300,-82.404031
540,100% Corpus,38,0.01,0.01,-17.429755,-82.404031
0,75% Corpus,30,0.01,0.01,-17.422493,-82.357875
300,100% Corpus,30,0.01,0.01,-17.589745,-82.357875
60,75% Corpus,32,0.01,0.01,-17.460892,-82.311669
...,...,...,...,...,...,...
238,75% Corpus,37,asymmetric,0.91,-14.648421,-8.565073
178,75% Corpus,35,asymmetric,0.91,-13.165466,-8.560991
478,100% Corpus,35,asymmetric,0.91,-13.312777,-8.560991
388,100% Corpus,32,asymmetric,0.91,-13.368993,-8.559482


In [41]:
topic_num = model_res_sorted.iloc[0, model_res_sorted.columns.get_loc('Topics')]
alpha_val = model_res_sorted.iloc[0, model_res_sorted.columns.get_loc('Alpha')]
beta_val = model_res_sorted.iloc[0, model_res_sorted.columns.get_loc('Beta')]

print(f'determined number of topics: {topic_num}\n')
print(f'determined alpha value: {alpha_val}\n')
print(f'determined beta value: {beta_val}')

determined number of topics: 39

determined alpha value: 0.9099999999999999

determined beta value: 0.01


In [46]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus: Term Document Frequency
#corpus = [id2word.doc2bow(text) for text in data_lemmatized]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=gensim_corpus,
                                           id2word=id2word,
                                           num_topics=topic_num, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha_val,
                                           iterations=100,
                                           eta = beta_val,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(7,
  '0.287*"set" + 0.151*"branch" + 0.089*"technology" + 0.053*"program" + '
  '0.025*"ethic" + 0.007*"go" + 0.000*"reflexion" + 0.000*"isometry" + '
  '0.000*"probability" + 0.000*"teaching"'),
 (2,
  '0.321*"study" + 0.098*"consider" + 0.097*"early" + 0.067*"generally" + '
  '0.000*"hyperplane" + 0.000*"probability" + 0.000*"namely" + 0.000*"attach" '
  '+ 0.000*"teaching" + 0.000*"scholarly"'),
 (14,
  '0.257*"datum" + 0.184*"see" + 0.121*"value" + 0.082*"professional" + '
  '0.028*"recent" + 0.000*"hyperplane" + 0.000*"probability" + 0.000*"namely" '
  '+ 0.000*"attach" + 0.000*"teaching"'),
 (27,
  '0.201*"example" + 0.135*"application" + 0.132*"role" + 0.088*"nature" + '
  '0.029*"broad" + 0.026*"contrast" + 0.015*"computing" + 0.000*"namely" + '
  '0.000*"teaching" + 0.000*"isometry"'),
 (1,
  '0.230*"method" + 0.125*"resource" + 0.106*"machine_learne" + 0.000*"base" + '
  '0.000*"refer" + 0.000*"namely" + 0.000*"reflection" + 0.000*"isometry" + '
  '0.000*"hyperplane" + 0.00

In [48]:
#calculating model perplexity
perplexity = lda_model.log_perplexity(gensim_corpus)

print(perplexity)
#lower the score, accurate is the model

-81.12560469839573


In [53]:
# Visualize the topics
'''
pyLDAvis.enable_notebook()
#vis = gensimvis.prepare(lda_model, corpus, id2word)
vis = gensimvis.prepare(lda_model, gensim_corpus, id2word, mds = "mmds", R=20)
vis'''

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.360920 -0.254769       1        1  2.734152
14    -0.275085 -0.080293       2        1  2.653820
18    -0.382944  0.244921       3        1  2.643846
25     0.172219 -0.240986       4        1  2.641624
6     -0.083823 -0.408649       5        1  2.635813
37     0.409306 -0.047252       6        1  2.628765
32     0.121454 -0.373574       7        1  2.626957
15    -0.356861  0.061004       8        1  2.620445
3     -0.096183  0.352883       9        1  2.616522
5      0.246207  0.307778      10        1  2.607425
20     0.285816 -0.262123      11        1  2.606440
11     0.309430  0.226389      12        1  2.595083
24     0.293935 -0.105130      13        1  2.594099
35    -0.176071 -0.247082      14        1  2.582955
22    -0.168817  0.175203      15        1  2.582750
36     0.060387  0.293064      16        1  2.576369
7     -0.183730  0.034317      17        1  2.575539
28    -0.369426 -0.069255      18        1  2.573032
19     0.336618  0.107016      19        1  2.554707
38    -0.046611  0.221301      20        1  2.551860
34    -0.178314  0.273442      21        1  2.549181
27     0.198565  0.076311      22        1  2.546383
4      0.041898  0.371989      23        1  2.544115
2     -0.068494  0.118073      24        1  2.542977
8      0.059574 -0.264274      25        1  2.539634
31    -0.190114 -0.162620      26        1  2.536470
33     0.270698  0.006279      27        1  2.531785
12     0.071251  0.148549      28        1  2.528324
0     -0.057512 -0.281464      29        1  2.527524
10     0.217650 -0.141838      30        1  2.526855
26     0.041571 -0.167803      31        1  2.525019
30    -0.244963  0.117925      32        1  2.521518
23     0.151677  0.234073      33        1  2.509419
1      0.080955  0.022403      34        1  2.497080
21    -0.069539 -0.165184      35        1  2.493119
29    -0.028107 -0.001671      36        1  2.482194
9     -0.110298 -0.059284      37        1  2.471264
17     0.062767 -0.057114      38        1  2.467162
16     0.015834 -0.002555      39        1  2.457777, topic_info=         Term        Freq       Total Category  logprob  loglift
203   include  309.000000  309.000000  Default  20.0000  20.0000
56     system  234.000000  234.000000  Default  19.0000  19.0000
528     large  215.000000  215.000000  Default  18.0000  18.0000
40      model  200.000000  200.000000  Default  17.0000  17.0000
246      also  185.000000  185.000000  Default  16.0000  16.0000
..        ...         ...         ...      ...      ...      ...
10      study    0.059552  153.768769  Topic39  -8.9489  -4.1504
11       well    0.059552   81.059290  Topic39  -8.9489  -3.5102
13      basic    0.059552   23.650738  Topic39  -8.9489  -2.2784
14       book    0.059552   78.139909  Topic39  -8.9489  -3.4735
15   describe    0.059552   39.518429  Topic39  -8.9489  -2.7918

[1055 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
124      16  0.960773     academic
65        7  0.882353     advanced
249      27  0.917346  agriculture
113      31  0.975669          aim
367       1  0.778727    algorithm
...     ...       ...          ...
64       18  0.837375          web
11       25  0.986932         well
471      27  0.988628         work
79        8  0.989360        world
238      25  0.957653        write

[212 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 15, 19, 26, 7, 38, 33, 16, 4, 6, 21, 12, 25, 36, 23, 37, 8, 29, 20, 39, 35, 28, 5, 3, 9, 32, 34, 13, 1, 11, 27, 31, 24, 2, 22, 30, 10, 18, 17])

In [55]:
def format_topics_sentences(ldamodel=None, corpus=gensim_corpus, texts=df):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=gensim_corpus, texts=data_lemmatized)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2,0.0472,"study, consider, early, generally, hyperplane,...","[metaphysic, ontology, philosophical, study, w..."
1,1,11,0.0701,"call, development, practice, technique, collec...","[guerilla, ontology, practice, describe, autho..."
2,2,36,0.0701,"process, come, analysis, academic, category, i...","[information, science, ontology, encompass, re..."
3,3,17,0.0484,"major, way, play, think, teaching, stem, hyper...","[gene, ontology, go, major, bioinformatic, ini..."
4,4,34,0.0719,"number, public, information, social, various, ...","[information, science, upper, ontology, top, l..."
5,5,37,0.0496,"know, form, learn, limit, integration, view, a...","[meta, ontology, study, field, inquiry, know, ..."
6,6,18,0.0472,"system, type, sometimes, domain, attribute, hy...","[system, engineering, ontology, engineering, f..."
7,7,8,0.0669,"language, well, launch, write, programming, bi...","[web, ontology, language, owl, family, knowled..."
8,8,24,0.0484,"make, product, start, tool, life, environment,...","[ontological, argument, philosophical, argumen..."
9,9,3,0.0460,"model, theory, event, need, committee, attach,...","[process, ontology, refer, universal, model, s..."


In [56]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)
    
# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0,0.0942,"produce, different, help, philosophy, reference, discipline, institute, attach, namely, hyperplane","[project, management, ready, discipline, agile, scrum_master, dasm, discipline, agile, senior, s..."
1,1,0.0920,"method, resource, machine_learne, base, refer, namely, reflection, isometry, hyperplane, probabi...","[machine_learne, field, inquiry, devote, understanding, building, method, learn, method, leverag..."
2,2,0.0737,"study, consider, early, generally, hyperplane, probability, namely, attach, teaching, scholarly","[discrete, study, mathematical, structure, consider, discrete, way, analogous, discrete, variabl..."
3,3,0.1299,"model, theory, event, need, committee, attach, probability, stem, namely, hyperplane","[mathematical, logic, study, formal, logic, mathematic, major, subarea, include, model, theory, ..."
4,4,0.1243,"history, state, location, influence, combine, reflection, hyperplane, probability, isometry, ref...","[history, timeline, russian, history, timeline, spanish_history, timeline, turkish, history, tim..."
5,5,0.1016,"include, may_refer, generation, food, attach, scholarly, hyperplane, probability, namely, reflec...","[scrum, first, generation, autozam, scrum, first, generation, autozam, scrum, turbo, first, gene..."
6,6,0.0990,"object, term, refer, thing, attach, reflection, isometry, hyperplane, probability, namely","[size, mathematical, object, property, determine, object, large, small, object]"
7,7,0.1156,"set, branch, technology, program, ethic, go, reflexion, isometry, probability, teaching","[ethic, branch, ethic, technology, specific, artificially, intelligent, system, sometimes, divide]"
8,8,0.0920,"language, well, launch, write, programming, biological, hyperplane, probability, attach, isometry","[torch, scientific, computing, framework, script, language, base, programming, language, provide]"
9,9,0.0776,"often, people, ground, scholarly, isometry, hyperplane, probability, namely, attach, teaching","[swiss, people, german, die, schweizer, french, suisse, italian, gli, romansh, il, svizzer, citi..."


In [57]:
df

,Unnamed: 0,id,key,title,excerpt,matched_title,description,thumbnail
0,0,22261,Ontology,Ontology,"metaphysics, ontology is the philosophical study of being, as well as related concepts such as e...",NaN,"Branch of philosophy that studies concepts such as existence, being, becoming, and reality","{'mimetype': 'image/png', 'size': None, 'width': 200, 'height': 137, 'duration': None, 'url': '/..."
1,1,2477036,Guerrilla_ontology,Guerrilla ontology,Guerilla ontology is a practice described by author Robert Anton Wilson in his 1980 book The Ill...,NaN,NaN,NaN
2,2,49681,Ontology_(information_science),Ontology (information science),"In computer science and information science, an ontology encompasses a representation, formal na...",NaN,Specification of a conceptualization,NaN
3,3,1454791,Gene_Ontology,Gene Ontology,The Gene Ontology (GO) is a major bioinformatics initiative to unify the representation of gene ...,NaN,Bioinformatics initiative,NaN
4,4,3200382,Upper_ontology,Upper ontology,"information science, an upper ontology (also known as a top-level ontology, upper model, or foun...",NaN,Ontology applicable across domains of knowledge,NaN
...,...,...,...,...,...,...,...,...
1795,1795,3858681,List_of_dog_diseases,List of dog diseases,"is a disease in dogs causing sudden blindness. It can occur in any breed. The cause is unknown, ...",NaN,NaN,NaN
1796,1796,22191094,Autoimmune_polyendocrine_syndrome_type_2,Autoimmune polyendocrine syndrome type 2,II is defined as the association between autoimmune Addison's disease and either autoimmune thyr...,NaN,Medical condition,"{'mimetype': 'image/png', 'size': None, 'width': 200, 'height': 195, 'duration': None, 'url': '/..."
1797,1797,5812984,Autoimmune_lymphoproliferative_syndrome,Autoimmune lymphoproliferative syndrome,needed] Autoimmune disease is the second most common clinical manifestation and one that most of...,NaN,Rare genetic medical disorder with abnormal lymphocyte survival,NaN
1798,1798,42233582,Antigen-antibody_interaction,Antigen-antibody interaction,triggers unexpected immune responses. This results in different autoimmune diseases depending on...,NaN,NaN,NaN


In [58]:
#df_dominant_topic.to_excel('df_dominant_topic.xlsx')
#sent_topics_sorteddf_mallet.to_excel('sent_topics_sorteddf_mallet.xlsx')
#df.to_excel('df.xlsx')

In [59]:
#df_with_topics = pd.merge(df, df_dominant_topic, how="left", on=["ip_address"])
df_with_topics = pd.concat([df, df_dominant_topic], axis=1).reindex(df.index)

In [62]:
#df_with_topics.to_excel('df_with_topics.xlsx')

In [ ]:
#df_with_topics = pd.read_excel("df_with_topics.xlsx")
data = df_with_topics.copy()
data['Reviews'] = data['excerpt']
data.head()

In [46]:
data = df.copy()
data['Reviews'] = data[str('excerpt')]
#data['Reviews'] = re.sub("[^a-zA-Z]",  # Search for all non-letters
#                          " ",          # Replace all non-letters with spaces
#                          data[str('excerpt')]

In [47]:
#YAKE
def yake_pipeline_monogram(x):
    language = 'en'
    max_ngram_size = 1
    deduplication_threshold = 0.9
    numOfKeywords = 5  # <- Multiple keywords
    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size,
                                            dedupLim=deduplication_threshold,
                                            top=numOfKeywords, features=None)
    #kw_extractor = yake.KeywordExtractor(top=5, stopwords=None)
    keywords_yake = kw_extractor.extract_keywords(x)
    return keywords_yake

def yake_pipeline_bigram(x):
    language = 'en'
    max_ngram_size = 2
    deduplication_threshold = 0.9
    numOfKeywords = 5  # <- Multiple keywords
    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size,
                                            dedupLim=deduplication_threshold,
                                            top=numOfKeywords, features=None)
    #kw_extractor = yake.KeywordExtractor(top=5, stopwords=None)
    keywords_yake = kw_extractor.extract_keywords(x)
    return keywords_yake
    
data['YAKE_mono'] =data['Reviews'].apply(lambda x: yake_pipeline_monogram(x))
data['YAKE_bi'] =data['Reviews'].apply(lambda x: yake_pipeline_bigram(x))

In [48]:
data.head()

,topic,keyword_extraction,excerpt,Reviews,YAKE_mono,YAKE_bi
0,Information Technology,not applicable,Knowing instructors have experiential knowledg...,Knowing instructors have experiential knowledg...,"[(instruction, 0.044949512008338896), (technol...","[(Knowing instructors, 0.01889972694060169), (..."
1,Information Technology,not applicable,This module consists of a PowerPoint presentat...,This module consists of a PowerPoint presentat...,"[(HEV, 0.049279661858695895), (systems, 0.1157...","[(module consists, 0.03071626848783736), (seed..."
2,Information Technology,not applicable,Geo-information has proven to be extremely he...,Geo-information has proven to be extremely he...,"[(disaster, 0.09456918960524358), (risk, 0.113...","[(damage detection, 0.0171166371983025), (situ..."
3,Information Technology,not applicable,Engineering and Technology for the Better Good...,Engineering and Technology for the Better Good...,"[(public, 0.031131261476839655), (applications...","[(Project Management, 0.004704193565275322), (..."
4,Information Technology,not applicable,Mushroom cultivation technology course provide...,Mushroom cultivation technology course provide...,"[(Mushroom, 0.10086407396398608), (information...","[(Mushroom cultivation, 0.045375978597222665),..."


In [49]:
#RAKE
r = Rake()    

def rake_implement(x,r):
    r.extract_keywords_from_text(x)
    return r.get_ranked_phrases_with_scores()

data['RAKE'] =data['Reviews'].apply(lambda x: rake_implement(x,r))


def rake_implement_phrases(x,r):
    r.extract_keywords_from_text(x)
    return r.get_ranked_phrases()
    
data['RAKE_phrases'] =data['Reviews'].apply(lambda x: rake_implement_phrases(x,r))

In [50]:
#Summa
def summa_pipeline(text):
    TR_keywords = keywords.keywords(text, scores=True)
    return TR_keywords[0:10]

def summa_pipeline_no_scores(text):
    TR_keywords = keywords.keywords(text, scores=False)
    return TR_keywords[0:10]

data['summa'] = data["Reviews"].apply(summa_pipeline)
data['summa_no_scores'] = data["Reviews"].apply(summa_pipeline_no_scores)

In [53]:
#KeyBert
def keybert_pipeline_mono(text):
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(1, 1),
                                     #keyphrase_ngram_range=(2, 2),
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)

    return keywords

def keybert_pipeline_bi(text):
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(2, 2),
                                     #keyphrase_ngram_range=(1, 1),
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)

    return keywords

def keybert_pipeline_mono_no_scores(text):
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(1, 1),
                                     #keyphrase_ngram_range=(2, 2),
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)
    keywords_list= list(dict(keywords).keys())

    return keywords_list

def keybert_pipeline_bi_no_scores(text):
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(2, 2),
                                     #keyphrase_ngram_range=(1, 1),
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)
    keywords_list= list(dict(keywords).keys())

    return keywords_list

In [54]:
data['KeyBert_mono'] = data["Reviews"].apply(keybert_pipeline_mono)
data['KeyBert_bi'] = data["Reviews"].apply(keybert_pipeline_bi)
data['KeyBert_mono_no_scores'] = data["Reviews"].apply(keybert_pipeline_mono_no_scores)
data['KeyBert_bi_no_scores'] = data["Reviews"].apply(keybert_pipeline_bi_no_scores)

In [52]:
#Keyword extraction from NLTK
stop_words = stopwords.words('english')

def get_keywords(row):
    some_text = row['Reviews']
    lowered = some_text.lower()
    tokens = nltk.tokenize.word_tokenize(some_text)
    keywords = [keyword for keyword in tokens if keyword.isalpha() and not keyword in stop_words]
    keywords_string = ','.join(keywords)
    return keywords_string

#replaces all occurring digits in the strings with nothing
#data['Reviews'] = data['Reviews'].str.replace('\d+', '')
#drop all the nan values 
#data=data.dropna()
#if you need the whole row to be passed inside the function
data['Keywords_nltk'] = data.apply(lambda row:get_keywords(row), axis=1)

#data['Keywords'] = data['Reviews'].apply(get_keywords, axis=1) 

In [55]:
data['rake_cleaned'] = data['RAKE_phrases'].str.join(', ')
data['keybert_mono_cleaned'] = data['KeyBert_mono_no_scores'].str.join(', ')
data['keybert_bi_cleaned'] = data['KeyBert_bi_no_scores'].str.join(', ')

data['level-3'] = data['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[0])

data['level-4'] = data['keybert_bi_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[:2])
data['level-4'] = data['level-4'].str.join(', ')
data["level-4"]=data["level-4"].str.replace(',','')

In [2]:
#data.to_excel('data with keywords new data.xlsx')
data = pd.read_excel("data with keywords new data.xlsx")

In [4]:
#zero shot classification
classifier_pipeline = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli")

In [7]:
# Geography and History

def zeroshotPipeline(text):
    input_ids = text
    #candidate_label = ['Ontology', 'Data Science', 'Astrophysics', 'Marine Biology', 'Climate Change', 'Psychology', 'War', 'Artificial Intelligence', 'Machine Learning', 'Scrum Master', 'Contemporary Art', 'Vegan Cooking', 'Landscape Photography', 'Switzerland', 'European Union', 'NATO', 'Family Constellations', 'Bioengineering', 'Medicine', 'Physics', 'Mathematics', 'Data Visualization', 'Topic Modeling', 'Antarctica', 'Sequoia', 'Blue Whale', 'Matcha', 'Solar Panels', 'Thyroid', 'Gender equality', 'Education', 'Developing country', 'Spanish History', 'Dog training', 'Solar System', 'Autoimmune disease']
    candidate_label = ['Geography',
                        'History',
                        'art',
                        'civilization',
                        'science',
                        'cultural',
                        'environmental',
                        'colonial',
                        'planet',
                        'others']

    results = classifier_pipeline(input_ids, 
                         candidate_label)
                        # multi_label = True)
                        # hypothesis_template = template)
    score_id = np.argmax(results["scores"])
    final_label = results["labels"][score_id]
    prob = results["scores"][score_id]
    return final_label, prob


Geoandhist = data.loc[data['topic'] == 'Geography and History']
Geoandhist["zero_shot_class_1"] = ""
Geoandhist['zero_shot_class_1'] = Geoandhist["Reviews"].apply(zeroshotPipeline)
split_geoandhist = pd.DataFrame(Geoandhist['zero_shot_class_1'].tolist(), columns=['level-1', 'level-1 score'])
Geoandhist.reset_index(drop=True, inplace=True)
split_geoandhist.reset_index(drop=True, inplace=True)
data1 = pd.concat([Geoandhist, split_geoandhist], axis=1)
geoandhist = data1.copy()
geoandhist.to_excel('geoandhist.xlsx')

In [16]:
# Robotics

def zeroshotPipeline(text):
    input_ids = text
    #candidate_label = ['Ontology', 'Data Science', 'Astrophysics', 'Marine Biology', 'Climate Change', 'Psychology', 'War', 'Artificial Intelligence', 'Machine Learning', 'Scrum Master', 'Contemporary Art', 'Vegan Cooking', 'Landscape Photography', 'Switzerland', 'European Union', 'NATO', 'Family Constellations', 'Bioengineering', 'Medicine', 'Physics', 'Mathematics', 'Data Visualization', 'Topic Modeling', 'Antarctica', 'Sequoia', 'Blue Whale', 'Matcha', 'Solar Panels', 'Thyroid', 'Gender equality', 'Education', 'Developing country', 'Spanish History', 'Dog training', 'Solar System', 'Autoimmune disease']
    candidate_label = ['lego',
                        'mindstorm',
                        'robotics',
                        'technology',
                        'cinema',
                        'motors',
                        'programming',
                        'geometry',
                        'batteries',
                        'others']

    results = classifier_pipeline(input_ids, 
                         candidate_label)
                        # multi_label = True)
                        # hypothesis_template = template)
    score_id = np.argmax(results["scores"])
    final_label = results["labels"][score_id]
    prob = results["scores"][score_id]
    return final_label, prob


Robotics = data.loc[data['topic'] == 'Robotics']
Robotics["zero_shot_class_1"] = ""
Robotics['zero_shot_class_1'] = Robotics["Reviews"].apply(zeroshotPipeline)
split_robotics = pd.DataFrame(Robotics['zero_shot_class_1'].tolist(), columns=['level-1', 'level-1 score'])
Robotics.reset_index(drop=True, inplace=True)
split_robotics.reset_index(drop=True, inplace=True)
data1 = pd.concat([Robotics, split_robotics], axis=1)
robotics = data1.copy()
robotics.to_excel('robotics.xlsx')

In [6]:
data.to_excel('data with keywords and zero shot new data.xlsx')

In [30]:
#traf_dist = logs3.loc[logs3['Insight Type'] == 'traffic distribution']
#df.drop(['C', 'D'], axis=1)

data_m = pd.read_excel("geoandhist_m.xlsx")

In [31]:
data_m['kw_1'] = data_m['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[0])
data_m['kw_2'] = data_m['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[1])
data_m['kw_3'] = data_m['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[2])
data_m['kw_4'] = data_m['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[3])
data_m['kw_5'] = data_m['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[4])

In [32]:
data_m1 = data_m.drop(['Unnamed: 0.1', 'Unnamed: 0', 'keyword_extraction', 'Reviews', 'YAKE_mono', 'YAKE_bi', 'RAKE', 'RAKE_phrases', 'summa', 'summa_no_scores', 'Keywords_nltk', 'KeyBert_mono', 'KeyBert_bi', 'KeyBert_bi_no_scores', 'rake_cleaned', 'KeyBert_mono_no_scores', 'keybert_bi_cleaned', 'level-3', 'level-4', 'zero_shot_class_1', 'level-1 score', 'keybert_mono_cleaned'], axis=1)

In [33]:
data_m1.head()

,topic,excerpt,level-1,kw_1,kw_2,kw_3,kw_4,kw_5
0,Geography and History,The course provides students with 1 an introdu...,science,geology,geologic,geological,field,introductory
1,Geography and History,This video was recorded at YALE RLST 145 - Int...,History,deuteronomy,prophets,testament,covenant,bible
2,Geography and History,This video was recorded at MIT World Series: T...,History,africa,ghana,globalization,african,africas
3,Geography and History,"Spelling games, math games and quizzes that le...",others,quizzes,spell,games,game,spelling
4,Geography and History,The ATETV project delivers web-based videos to...,History,atetv,gis,maps,project,technology


In [34]:
gandh = data_m1.copy()

In [28]:
rob.head()

,topic,excerpt,level-1,kw_1,kw_2,kw_3,kw_4,kw_5
0,Robotics,This video was recorded at MIT World: One Host...,others,china,globalization,mit,zhou,chinese
1,Robotics,This video was recorded at MIT World: One Host...,technology,grants,funding,entrepreneurs,grant,government
2,Robotics,A Lego Robotics Stand-Alone activity designed ...,robotics,lego,robotics,activities,activity,introductory
3,Robotics,This is a 1:09 minute long YouTube video of a ...,programming,robotics,rubiks,mindstorms,lego,robot
4,Robotics,This video was recorded at European Conference...,robotics,kernelization,kernelized,sparsification,kernel,sparse


In [35]:
final = [rob, gandh]

In [37]:
result = pd.concat(final)

In [38]:
result.head()

,topic,excerpt,level-1,kw_1,kw_2,kw_3,kw_4,kw_5
0,Robotics,This video was recorded at MIT World: One Host...,others,china,globalization,mit,zhou,chinese
1,Robotics,This video was recorded at MIT World: One Host...,technology,grants,funding,entrepreneurs,grant,government
2,Robotics,A Lego Robotics Stand-Alone activity designed ...,robotics,lego,robotics,activities,activity,introductory
3,Robotics,This is a 1:09 minute long YouTube video of a ...,programming,robotics,rubiks,mindstorms,lego,robot
4,Robotics,This video was recorded at European Conference...,robotics,kernelization,kernelized,sparsification,kernel,sparse


In [39]:
result.shape

(329, 8)

In [40]:
rob.shape

(151, 8)

In [41]:
gandh.shape

(178, 8)

In [44]:
#result.to_csv('data.csv')

result.to_excel('data_excel.xlsx')